# INSTALLS

In [ ]:
!pip install crewai crewai_tools langchain_community

In [ ]:
#!pip install CrewAI crewai_tools langchain_community

In [ ]:
#!pip install pandas openpyxl

In [ ]:
#!pip install langchain-groq

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import json
from crewai import Agent, Task, Crew, Process

In [ ]:
import os
from google.colab import userdata


# os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
#os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
# os.environ["OPENAI_API_KEY"] = "placeholder_key"






os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY') #was "patents api key"
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
os.environ["EXASEARCH_API_KEY"] = userdata.get('EXASEARCH_API_KEY')


In [ ]:
# from dotenv import load_dotenv
# from langchain_groq import ChatGroq

# from crewai import LLM

# llm = LLM(
#     model="groq/llama-3.1-8b-instant"
# )

# TOOLS

In [ ]:
# # Creating tools
!pip install requests  # Install the requests library
import requests

In [ ]:
# Patent search tool NEW.

import json
import os
import requests
from crewai_tools import BaseTool

class PatentSearchTool(BaseTool):
    name: str = "Patent Search tool"
    description: str = "Search the internet for Patents."

    # @record_tool(tool_name="Patent search tool")
    def _run(self, query: str) -> str:
        """
        Search the internet for Patents.
        """

        url = "https://google.serper.dev/patents"

        payload = json.dumps({
            "q": query,
            "num": 2,
            "autocorrect": False,
            "tbs": "qdr:d"
        })

        headers = {
            'X-API-KEY': os.getenv('SERPER_API_KEY'),
            'Content-Type': 'application/json'
        }

        response = requests.request("POST", url, headers=headers, data=payload)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the JSON response
            response_data = response.json()

            # Log the entire response structure for debugging
            print("Response data structure:", json.dumps(response_data, indent=2))

            # Ensure 'figures' exists and is a list, and limit to 1 figure
            if 'figures' in response_data and isinstance(response_data['figures'], list):
                if response_data['figures']:  # Check if the list is not empty
                    response_data['figures'] = [response_data['figures'][0]]  # Keep only the first figure
                else:
                    response_data['figures'] = []  # If no figures, set to an empty list
            else:
                response_data['figures'] = []  # If 'figures' key does not exist, set to an empty list

            # Ensure only one URL and thumbnail, assuming they are nested under 'results' (adjust based on actual API response)
            if 'results' in response_data and isinstance(response_data['results'], list) and response_data['results']:
                first_result = response_data['results'][0]  # Get the first result only
                response_data['results'] = [first_result]  # Replace results with only the first result

                # Optionally, if URLs and thumbnails are within each result
                if 'url' in first_result:
                    first_result['url'] = first_result['url']
                if 'thumbnail' in first_result:
                    first_result['thumbnail'] = first_result['thumbnail']

            # Convert the response data to a JSON string and return
            return json.dumps(response_data, indent=2)

        # Handle non-200 status code
        return f"Error: Received status code {response.status_code}"

In [ ]:
# search scholar

class ScholarSearchTool(BaseTool):
    name: str = "Scholar search tool"
    description: str = "Search the internet for academic articles."

    # @record_tool(tool_name="Scholar Search Tool")
    def _run(self, query: str) -> str:
        """
        Search the internet for academic articles.
        """

        url = "https://google.serper.dev/scholar"

        payload = json.dumps({
            "q": query,
            "num": 2,
            "autocorrect": False,
            "tbs": "qdr:d"
        })

        headers = {
            'X-API-KEY': os.getenv('SERPER_API_KEY'),
            'Content-Type': 'application/json'
        }

        response = requests.request("POST", url, headers=headers, data=payload)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the JSON response
            response_data = response.json()

        # Convert the news data back to a JSON string
        return json.dumps(response_data, indent=2)

In [ ]:
# search news

class NewsSearchTool(BaseTool):
    name: str = "Custom Serper Dev Tool"
    description: str = "Search the internet for news."

    # @record_tool(tool_name="Custom Serper Dev Tool")
    def _run(self, query: str) -> str:
        """
        Search the internet for news.
        """

        url = "https://google.serper.dev/news"

        payload = json.dumps({
            "q": query,
            "num": 5,
            "autocorrect": False,
            "tbs": "qdr:d"
        })

        headers = {
            'X-API-KEY': os.getenv('SERPER_API_KEY'),
            'Content-Type': 'application/json'
        }

        response = requests.request("POST", url, headers=headers, data=payload)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the JSON response
            response_data = response.json()

            # Log the entire response structure for debugging
            print("Response data structure:", json.dumps(response_data, indent=2))

            # Ensure 'news' exists and is a list
            if 'news' in response_data and isinstance(response_data['news'], list):
                if response_data['news']:  # Check if the list is not empty
                    response_data['news'] = [response_data['news'][0]]  # Keep only the first news item
                else:
                    response_data['news'] = []  # If no news items, set to an empty list
            else:
                response_data['news'] = []  # If 'news' key does not exist, set to an empty list

            # Convert the response data to a JSON string
            return json.dumps(response_data, indent=2)

        # Handle non-200 status code
        return f"Error: Received status code {response.status_code}"

In [ ]:

# Custom file writer function
def write_to_file(content, directory='project', filename='output.txt'):
    if not os.path.exists(directory):
        os.makedirs(directory)

    file_path = os.path.join(directory, filename)

    # Ensure content is a string before writing
    if not isinstance(content, str):
        content = str(content)

    with open(file_path, 'w') as file:
        file.write(content)

    print(f"Content written to {file_path}")

In [ ]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool, \
                         DallETool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool, \
                         EXASearchTool


#Dalle tool is actually used. Vision Tool not working

In [ ]:
# #Access Google Drive to write files
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
#Reading files

#read_inputs = DirectoryReadTool(directory='/content/drive/MyDrive/Creative_AI/research_engine/Inputs')
#read_outputs = DirectoryReadTool(directory='/content/drive/MyDrive/Creative_AI/research_engine/Outputs')
#read_baseline= FileReadTool(file_path='/content/drive/MyDrive/Creative_AI/research_engine/Outputs/knowledge_baseline.txt')
#file_read_tool = FileReadTool()

In [ ]:
# Direct file reading using Python standard library
#file_path = '/content/drive/MyDrive/Creative_AI/research_engine/Inputs/test.txt'
#with open(file_path, 'r') as file:
#    content = file.read()

#Check if content is being read
#print(content)


In [ ]:
#instantiating other tools

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
website_search_tool = WebsiteSearchTool()
exa_search_tool = EXASearchTool()

#vision_tool = VisionTool(model="dall-e-3",
#                       size="1024x1024",
#                       quality="standard",
#                       n=1)

# AGENTS

In [ ]:
# #not used
# manager = Agent(
#     role="Project Manager",
#     goal="Efficiently manage the research team and ensure the production of world-class research reports",
#     backstory=(
#         "You are a highly experienced research project manager, overseeing a team dedicated to producing exceptional research reports. "
#         "Your responsibilities include ensuring quality at each stage: knowledge collection, analysis, interpretation, and reporting. "
#         "Implement multiple revision loops, verifying each stage for accuracy, completeness, and relevance to the topic, purpose, and context."
#         "\n\nInstructions:\n"
#         "1. **Knowledge Collection**: Begin by collecting all relevant knowledge on the {topic} within the {context}. Gather only reliable and relevant sources.\n"
#         "2. **Analysis & Interpretation**: Conduct a detailed analysis, identifying key trends, insights, and implications. Ensure interpretations are accurate and based on the collected data.\n"
#         "3. **Quality Checks**: After each stage, evaluate completeness, accuracy, and relevance to {purpose}. If content is lacking, instruct the team to revise it accordingly.\n"
#         "4. **Reporting**: Guide the team in drafting a final report. Structure it clearly with an introduction, analysis, and conclusion sections.\n"
#         "If any stage is incomplete or fails to meet standards, delegate tasks back to team members for further improvement.\n"
#         "Research topic: {topic}\n"
#         "Research purpose: {purpose}\n"
#         "Research context: {context}\n"
#     ),
#     # llm=llm,
#     allow_delegation=True,
#     verbose=True,
#     memory=True,
#     # max_iter=2,  # Uncomment to limit iterations for efficiency
# )


In [ ]:
strategist_agent = Agent(
    role="Strategist",
    goal="Analyze and unpack the challenge to reveal its core components and actionable steps",
    backstory=(
        "As an expert strategist, you excel at breaking down complex challenges into clear, manageable elements. "
        "Your goal is to analyze the challenge thoroughly, identifying core issues, key factors, and any underlying assumptions. "
        "From this analysis, outline a clear strategy, highlighting priority areas and actionable steps to address the challenge effectively."
    ),
    instructions=(
        "1. **Identify Core Elements**: Begin by identifying and listing the main components of the challenge. "
        "Break down complex issues into their constituent parts to make them easier to understand.\n"
        "2. **Analyze Key Factors**: Examine each core element, identifying any factors that impact the challenge, such as constraints, risks, and opportunities.\n"
        "3. **Outline Assumptions**: Highlight any assumptions or uncertainties that might influence the approach to solving the challenge.\n"
        "4. **Recommend Actionable Steps**: Conclude by recommending actionable steps based on the analysis, prioritizing actions that are likely to have the highest impact.\n"
    ),
    # llm=llm,
    allow_delegation=False,
    # verbose=True,
    memory=True,
    # max_iter=2  # Uncomment to limit the iterations for efficiency
)


In [ ]:
researcher_agent = Agent(
    role="Researcher",
    goal="Locate and summarize relevant information to answer questions accurately and comprehensively",
    backstory=(
        "As a diligent and methodical researcher, you are tasked with gathering accessible and highly relevant information "
        "using the tools available. You excel at sifting through large amounts of information, extracting key points, "
        "and presenting them concisely."
        "\n\nInstructions:\n"
        "1. **Identify Relevant Sources**: Focus on finding credible and authoritative sources that address the question directly.\n"
        "2. **Extract Key Points**: From each source, extract only the most relevant details. Avoid overly general information or unrelated data.\n"
        "3. **Summarize Effectively**: Summarize findings in a clear, concise manner. If there are multiple aspects to the question, organize responses by topic.\n"
        "4. **Verify for Completeness**: Ensure that all aspects of the question are addressed comprehensively. If a gap remains, indicate further research areas.\n"
    ),
    # llm=llm,
    allow_delegation=False,
    # verbose=True,
    memory=True,
    # max_iter=4  # Uncomment to limit iterations for efficiency
)


In [ ]:
# #not used
# scraper_agent = Agent(
#     role="Scraper",
#     goal="Extract and structure key details from patent documents efficiently",
#     backstory=(
#         "As an expert scraper, you are skilled at extracting and organizing detailed information from patent documents. "
#         "Your role is to retrieve abstracts, claims, technical descriptions, and other relevant information with precision, "
#         "ensuring accuracy for subsequent analysis and reporting."
#     ),
#     instructions=(
#         "1. **Identify Key Sections**: Locate and extract the patent's abstract, claims, and technical descriptions. "
#         "Pay particular attention to details relevant to the invention's novelty, application, and technical aspects.\n"
#         "2. **Structure Data**: Organize extracted information into a structured format, separating sections such as abstract, claims, inventor information, and classifications.\n"
#         "3. **Check for Completeness**: Verify that each key section (abstract, claims, and technical descriptions) is captured accurately and is free from omissions.\n"
#         "4. **Flag Issues**: If any section is missing or lacks clarity, indicate these issues clearly for further review.\n"
#     ),
#     # llm=llm,
#     allow_delegation=False,
#     verbose=True,
#     memory=True,
#     # max_iter=2  # Uncomment to limit iterations for efficiency
# )


In [ ]:
writer_agent = Agent(
    role="Writer",
    goal="Summarize technical information from patents into clear, accessible formats for a broad audience",
    backstory=(
        "As a skilled technical writer, you excel at translating complex technical information from patents into concise and "
        "understandable summaries. You focus on key innovations, core claims, and potential applications, ensuring that both technical "
        "and non-technical audiences can grasp the main points effectively."
    ),
    instructions=(
        "1. **Identify Key Innovations**: Begin by summarizing the main innovation of the patent. Focus on what sets it apart and "
        "why it’s significant in the field.\n"
        "2. **Summarize Core Claims**: Highlight the primary claims, explaining the unique aspects of the invention and any technical advantages.\n"
        "3. **Outline Potential Applications**: Describe potential uses and applications of the patent, tailoring language to ensure accessibility for non-technical readers.\n"
        "4. **Use Clear, Concise Language**: Avoid overly technical jargon unless absolutely necessary. Where technical terms are used, provide brief clarifications if they might be unfamiliar to general readers.\n"
        "5. **Maintain Accuracy and Neutral Tone**: Ensure that summaries remain factual and neutral, focusing on the patent content without adding subjective interpretations.\n"
    ),
    # llm=llm,
    allow_delegation=False,
    verbose=1,
    memory=True,
    # max_iter=2  # Uncomment to limit iterations for efficiency
)


In [ ]:
insight_agent = Agent(
    role="Insight Strategist",
    goal="Generate innovative, unexpected insights by applying diverse mental models and angles to the problem",
    backstory=(
        "You are an extremely smart insight strategist with a knack for uncovering unique, out-of-the-box perspectives. "
        "While you have expertise in the sector of {sector}, you are proficient in integrating knowledge and methodologies from various disciplines. "
        "You excel at applying creative problem-solving techniques and mental models to generate fresh insights that challenge conventional thinking."
        "You also see insights from given documents in {doc_summary} if provided than generate final five insights."
    ),
    instructions=(
        "1. **Analyze from Multiple Angles**: Start by viewing the problem from several distinct perspectives. Apply frameworks from different disciplines (e.g., psychology, economics, history, technology) to uncover novel insights.\n"
        "2. **Use Creative Thinking Techniques**: Leverage creativity tools like lateral thinking, mind mapping, or reverse engineering to challenge assumptions and identify unconventional connections.\n"
        "3. **Look for Unexpected Connections**: Identify patterns or relationships that might not be immediately obvious. Cross-pollinate ideas from unrelated fields to produce original perspectives.\n"
        "4. **Identify Potential Impacts**: For each insight, consider its potential implications. How could it reshape current thinking or lead to unexpected breakthroughs?\n"
        "5. **Contextualize with Sector Knowledge**: Use your sector expertise to ground your insights, but don't be afraid to stretch the boundaries and consider cross-industry trends and innovations.\n"
        "6. **Encourage Provocative Ideas**: Aim to generate insights that challenge the status quo, provoke thought, and inspire new ways of thinking.\n"
        "7. **Keep Insights Actionable**: While creativity is important, ensure that each insight is actionable, offering practical value or guiding strategic decisions.\n"
    ),
    # llm=llm,
    allow_delegation=False,
    verbose=1,
    memory=True,
    # max_iter=2  # Uncomment to limit iterations for efficiency
)


## New Agents

In [ ]:
from langchain_openai import ChatOpenAI

opportunities_strategist = Agent(
    role="Opportunities Strategist",
    goal="Generate opportunity spaces that address the challenges and situation on the Company",
    backstory=(
        "Based on the insights previously generated, generate five opportunity spaces that address the challenge below based on the specific situation on the company. "
        "An opportunity space is not a specific solution, but a strategic or tactical direction which could be taken to address the challenge, leveraging the knowledge accrued in the insights."
    ),
    instructions=(
        "1. **Understand the Challenge**: Begin by deeply analyzing the specific challenge faced by the company, considering the context and the nuances of the situation described.\n"
        "2. **Leverage Existing Insights**: Use the insights already generated to inform your thinking. These insights should guide your understanding of the company's strengths, weaknesses, and external opportunities.\n"
        "3. **Frame Opportunity Spaces**: Create five opportunity spaces, each representing a strategic or tactical direction the company can explore. Avoid specific solutions; focus on broader directions that address the challenge effectively.\n"
        "4. **Incorporate Multidisciplinary Frameworks**: Use mental models and frameworks from diverse disciplines such as psychology, technology, and business strategy to ensure a comprehensive perspective.\n"
        "5. **Align with Company Goals**: Ensure each opportunity space aligns with the company's mission, values, and overarching objectives.\n"
        "6. **Assess Innovation Potential**: Evaluate the potential for innovation in each opportunity space. Identify how it could help the company achieve differentiation or competitive advantage.\n"
        "7. **Cross-Functional Considerations**: Analyze how each opportunity space impacts various functions or departments within the company. Highlight synergies or areas requiring alignment.\n"
        "8. **Make Actionable Suggestions**: Refine each opportunity space to ensure it is actionable. Clearly define how the company might begin exploring each direction."
    ),
    llm=ChatOpenAI(temperature=0.7, model="o1-preview"),
    allow_delegation=False,
    verbose=1,
    memory=True,
    # max_iter=2  # Uncomment to limit iterations for efficiency
)


In [ ]:
from crewai_tools import DallETool
dalle_tool = DallETool()

opp_images = Agent(
    role="Images Creator",
    goal="Generate images based on descriptions",
    backstory=(
        "An AI assistant specialized in creating visual content. Your primary task is to **Generate** visually appealing, realistic, and brand-aligned images based on the provided descriptions. "
        "Each image should reflect the essence of the opportunity space present in {opportunity_spaces} and adhere to the brand's DNA."
    ),
    instructions=(
        "1. **Analyze the Description**: Carefully read and understand the description of each opportunity space and the {opportunity_spaces} provided.\n"
        "2. **Maintain Brand Alignment**: Ensure the generated image aligns with the brand's DNA, focusing on elegance, simplicity, and relevance.\n"
        "3. **Generate Images**: Use the provided tools to create five images, one for each opportunity space, based on the first idea mentioned.\n"
        "4. **Adhere to Contextual Constraints**: Incorporate the sector, target clients, resources, strategic priorities, project name, challenge description, purpose, and focus constraints into the image generation process.\n"
        "5. **Ensure Visual Quality**: Each image should be simple, elegant, and essential while staying true to the brand."
    ),
    llm=ChatOpenAI(temperature=0.7, model_name="gpt-4-1106-preview"),
    allow_delegation=False,
    verbose=1,
    memory=True,
    tools=[dalle_tool]
)


In [ ]:
from crewai import Agent
from crewai_tools import CSVSearchTool, DOCXSearchTool, TXTSearchTool

doc_analyst = Agent(
    role="Document Analyzer",
    goal=(
        "1. Analyze the provided documents ({documents}) to extract relevant insights that address the specified challenge."
        "2. Focus on identifying actionable information and key elements from the content that could contribute to solving the challenge."
        "3. Highlight important details, including direct quotes, references, and their connections to the context outlined below."
     ),
    backstory=(
        "Analyze the attached documents {documents} to identify insights that could be relevant to finding solutions to the challenge below.  Make sure to include direct quotes from the text and references when relevant."
        "Output a summary of the content of the document, with particular focus on the elements that could help solve the challenge. If documents {documents} are not attached you should move on on another task. you should answer in english"

            "Context:\n"
            "Sector: {sector}\n"
            "Target Clients: {target_clients}\n"
            "Resources: {resources}\n"
            "Strategic Priorities: {strategic_priorities}\n"
            "Project Name: {project_name}\n"
            "Challenge Description: {challenge_description}\n"
            "Purpose: {purpose}\n"
            "Focus Constraints: {focus_constraints}"

    ),
    instructions=(
        "1. Carefully review the attached documents ({documents}) and identify insights that are relevant to the challenge.\n"
        "2. Summarize the content with a focus on the following contextual elements:\n"
        "   - Sector: {sector}\n"
        "   - Target Clients: {target_clients}\n"
        "   - Resources: {resources}\n"
        "   - Strategic Priorities: {strategic_priorities}\n"
        "   - Project Name: {project_name}\n"
        "   - Challenge Description: {challenge_description}\n"
        "   - Purpose: {purpose}\n"
        "   - Focus Constraints: {focus_constraints}\n"
        "3. Ensure that the summary includes actionable insights, direct quotes, and references to the text when relevant.\n"
        "4. If no documents ({documents}) are attached, you may skip this task and move on.\n"
        "5. Deliver the summary in a structured format that addresses the purpose of the challenge."),
    allow_delegation=False,
    verbose=1,
    memory=True,
    tools = [CSVSearchTool(),DOCXSearchTool(),TXTSearchTool()]
)

# TASKS

In [ ]:
generate_queries_task = Task(
    description=(
        "Based on the provided challenge (e.g., 'biodegradable packaging for food'), generate a set of five related search keywords "
        "that are concise, semantically related, and cover possible variations of the original query. "
        "The queries should be brief and use as few words as possible to maximize relevance and clarity.\n\n"

        "Context:\n"
        "Sector: {sector}\n"
        "Target Clients: {target_clients}\n"
        "Resources: {resources}\n"
        "Strategic Priorities: {strategic_priorities}\n"
        "Project Name: {project_name}\n"
        "Challenge Description: {challenge_description}\n"
        "Purpose: {purpose}\n"
        "Focus Constraints: {focus_constraints}\n\n"

        "Instructions:\n"
        "1. Identify the key concepts in the challenge description.\n"
        "2. Generate search queries that reflect these key concepts while considering variations in phrasing, synonyms, or related terms.\n"
        "3. Ensure that the queries are concise and directly related to the core aspects of the challenge.\n"
        "4. Aim for a balance of broad and specific keywords that would lead to relevant information.\n"
        "5. Ensure that each query is distinct but related, to maximize the range of results without redundancy."
    ),
    expected_output=(
        "A list of five keywords related to the challenge: {challenge_description} that can be used for further searching. "
        "Example: ['biodegradable food packaging', 'compostable packaging materials', 'eco-friendly packaging for food', 'plant-based packaging for perishables']"
    ),
    tools=[],
    agent=strategist_agent
)


## Patents and scholar

In [ ]:
patent_search_task = Task(
    description=(
        "For each of the queries generated in Task 1, use the search tool to query Google Patents and retrieve "
        "the top 5 most relevant search results for each query. Ensure that the patents selected are closely related to the original topic, "
        "focusing on titles, abstracts, and URLs. Avoid extracting image links; only text-based information should be extracted. "
        "Each query should be treated separately, ensuring the most relevant patents for each search term are identified."
    ),
    expected_output=(
        "A list of 25 total query results, combining the top 5 relevant patents for each of the 5 queries generated in Task 1. "
        "Each result should include the title, URL, and abstract, formatted in a tabular structure. Do not include images."
    ),
    tools=[PatentSearchTool()],
    agent=researcher_agent
)


In [ ]:
scholar_search_task = Task(
    description=(
        "For each of the queries generated in Task 1, use the search tool to query Google Scholar and retrieve "
        "the top 5 most relevant search results for each query. Ensure that the articles selected are closely related to the original topic, "
        "focusing on titles, abstracts, and URLs. Avoid extracting image links or full-text articles; only text-based information should be extracted. "
        "Each query should be treated separately to identify the most relevant academic articles for each search term."
    ),
    expected_output=(
        "A list of 25 total query results, combining the top 5 relevant academic articles for each of the 5 queries generated in Task 1. "
        "Each result should include the title, URL, and abstract, presented in a tabular format. Do not include images or full-text links."
    ),
    tools=[ScholarSearchTool()],
    agent=researcher_agent
)


In [ ]:
# Not used
# scrape_content_task = Task(
#     description=(
#         "For each of the top 5 patent results obtained in Task 2, use the scrape tool to capture the detailed content from the patent pages. "
#         "Ensure that the abstract, claims, and any relevant technical information are accurately captured for each patent. "
#         "Only extract text-based content—ignore images, diagrams, or any non-textual elements. "
#         "The focus should be on key textual information that provides insights into the patent’s claims, technical details, and innovations."
#     ),
#     expected_output=(
#         "A collection of the scraped content for each patent, presented in a structured format. "
#         "Each patent should include the abstract, claims, and any important technical details, with the text organized for easy analysis."
#     ),
#     tools=[scrape_tool],
#     agent=scraper_agent
# )


In [ ]:
select_patents_task = Task(
    description=(
        "Review all the patents identified in the previous task. For each patent, assess its relevance based on the "
        "provided context, including the challenge description, sector, target clients, and strategic priorities. "
        "Use your judgment to determine how closely the patent aligns with the core objectives of the project.\n\n"

        "Context:\n"
        "Sector: {sector}\n"
        "Target Clients: {target_clients}\n"
        "Resources: {resources}\n"
        "Strategic Priorities: {strategic_priorities}\n"
        "Project Name: {project_name}\n"
        "Challenge Description: {challenge_description}\n"
        "Purpose: {purpose}\n"
        "Focus Constraints: {focus_constraints}\n\n"

        "Instructions:\n"
        "1. Review the patent’s title, abstract, claims, and technical content.\n"
        "2. Evaluate how well the patent addresses the key challenges described in the original topic and purpose.\n"
        "3. Rank the patents in order of relevance based on how well they align with the project’s objectives.\n"
        "4. Provide a brief explanation of why each patent was selected, focusing on its relevance to the sector, the challenge, and strategic priorities."
    ),
    expected_output=(
        "A shortlist of the 10 most relevant patents to the original challenge: {challenge_description}. "
        "Present a table with the following columns: patent number, title, awardees, URL, a summary of the abstract, and a brief rationale explaining why each patent is relevant."
    ),
    agent=strategist_agent
)


In [ ]:
select_papers_task = Task(
    description=(
        "Review all the research papers identified in the previous task. For each paper, assess its relevance based on the "
        "provided context, including the challenge description, sector, target clients, and strategic priorities. "
        "Use your judgment to determine how closely the paper aligns with the core objectives of the project.\n\n"

        "Context:\n"
        "Sector: {sector}\n"
        "Target Clients: {target_clients}\n"
        "Resources: {resources}\n"
        "Strategic Priorities: {strategic_priorities}\n"
        "Project Name: {project_name}\n"
        "Challenge Description: {challenge_description}\n"
        "Purpose: {purpose}\n"
        "Focus Constraints: {focus_constraints}\n\n"

        "Instructions:\n"
        "1. Review the paper’s title, abstract, and key findings.\n"
        "2. Evaluate how well the paper addresses the key challenges outlined in the original topic and purpose.\n"
        "3. Rank the papers in order of relevance based on how well they align with the project’s objectives.\n"
        "4. Provide a brief explanation of why each paper was selected, focusing on its relevance to the sector, the challenge, and strategic priorities."
    ),
    expected_output=(
        "A shortlist of the 10 most relevant papers to the original challenge: {challenge_description}. "
        "Present a table with the following columns: paper title, authors, URL, summary of the abstract, and a brief rationale explaining why each paper is relevant."
    ),
    agent=strategist_agent
)


In [ ]:
summarize_task = Task(
    description=(
        "Based on the selected content from the previous step, first reproduce the output table from the previous task as it is, "
        "with the details of the patents and papers.\n\n"

        "Next, write a concise, clear, and comprehensive summary that explains the content of the selected patents and research papers. "
        "This summary should focus on how these findings relate to the original challenge and purpose. Ensure the summary includes the following:\n"
        "1. **Overview of Innovations**: Highlight the key technological innovations found in the papers and patents.\n"
        "2. **Research Centers and Trends**: Mention where major research is being conducted, which institutions or industries are leading in this space, and any significant geographical concentrations.\n"
        "3. **Research Age and Relevance**: Discuss how recent the research is and whether it's still relevant to the current challenge.\n"
        "4. **Technical Details Translation**: Translate technical content into digestible information for a non-technical audience, ensuring clarity and accessibility.\n"
        "5. **Relevance to Challenge**: Tie each piece of research to the challenge, explaining how it might contribute to solving the problem at hand."
    ),
    expected_output=(
        "Reproduce the output table from the previous task, followed by a clear and concise summary of the patents and papers. "
        "The summary should cover innovations, research trends, the age of research, and its relevance to the challenge."
    ),
    tools=[],  # Placeholder tool for summarizing content
    agent=writer_agent
)


## Other research tasks

In [ ]:
trends_task = Task(
    description=(
        "For the topic below, conduct research on market trends. Make sure to survey both demand and supply aspects. "
        "Do not only include aspects that are immediately related to the challenge, but also trends in the sector as a whole. "
        "Make sure to dig deep and go beyond the obvious to find interesting trends and weak signals. Be extensive, listing at least 10 items.\n\n"


        "Context:\n"
        "Sector: {sector}\n"
        "Target Clients: {target_clients}\n"
        "Resources: {resources}\n"
        "Strategic Priorities: {strategic_priorities}\n"
        "Project Name: {project_name}\n"
        "Challenge Description: {challenge_description}\n"
        "Purpose: {purpose}\n"
        "Focus Constraints: {focus_constraints}\n\n"

        "Keep in mind that who reads your output is already an expert on this sector, so make sure to use accurate, up-to-date sources, and come up with non-obvious results.\n"


    ),
    expected_output=(
        "Produce a list of at least 10 trends with the following structure:\n\n"
        "1. **Catchy Title**: Title that succinctly captures the essence of the trend.\n"
        "2. **Body**: A detailed discussion including:\n"
        "   - Description of the trend\n"
        "   - Main drivers of the trend\n"
        "   - Examples (mentioning relevant companies or products, where applicable)\n"
        "   - Links to sources for further reference."
    ),
    tools=[search_tool, scrape_tool],  # Placeholder tool for conducting research
    agent=researcher_agent
)


In [ ]:
# trends_task = Task(
#     description=(
#         "For the topic below, conduct research on market trends. Make sure to survey both demand and supply aspects. "
#         "Do not only include aspects that are immediately related to the challenge, but also trends in the sector as a whole. "
#         "Make sure to dig deep and go beyond the obvious to find interesting trends and weak signals. Be extensive, listing at least 10 items.\n\n"

#         "Here is an example of a good discussion of a trend:\n\n"

#         "TITLE: Are Beet Gummies the Next Big Thing in Functional Foods?\n\n"
#         "Body: Introduction\n"
#         "Functional foods have been steadily gaining popularity, with more consumers seeking products that offer health benefits beyond basic nutrition. "
#         "One of the latest entrants in this category is beet gummies. These chewy, nutrient-dense snacks are becoming increasingly popular for their health benefits and convenience. "
#         "But what exactly makes beet gummies so special, and why are they poised to become a staple in modern diets? Let's explore.\n\n"

#         "The Nutritional Powerhouse of Beets\n"
#         "Beets are well-known for their rich nutritional profile. Packed with essential vitamins and minerals such as vitamin C, potassium, and iron, beets are also a great source of dietary fiber. "
#         "What sets them apart in the realm of functional foods is their high nitrate content, which has been linked to improved blood flow and reduced blood pressure. "
#         "By incorporating these nutrients into a convenient gummy form, beet gummies offer a tasty and accessible way to boost one’s intake of these beneficial compounds.\n\n"

#         "Functional Foods: Beyond Basic Nutrition\n"
#         "Functional foods are designed to have a positive effect on health beyond basic nutrition. Beet gummies are a prime example of this trend. "
#         "They combine the inherent benefits of beets with the convenience of a gummy snack, making it easy for people to incorporate more nutrients into their diets without significant lifestyle changes. "
#         "This is particularly appealing in our fast-paced world where convenience and health often need to go hand-in-hand.\n\n"

#         "The Role of Beet Gummies in Modern Diets\n"
#         "In today's health-conscious society, consumers are more informed and mindful about what they eat. Beet gummies fit perfectly into this narrative by offering a delicious, easy-to-consume snack that supports overall wellness. "
#         "These gummies can be particularly appealing to those who might not enjoy the taste of raw or cooked beets but still want to reap their health benefits. "
#         "Incorporating beet gummies into daily routines can help improve dietary habits and contribute to better health outcomes over time.\n\n"

#         "Conclusion\n"
#         "The rise of beet gummies reflects a broader trend towards functional foods that provide more than just basic nutrition. Their combination of convenience, taste, and health benefits makes them a standout option for anyone looking to enhance their diet. "
#         "As the demand for functional foods continues to grow, beet gummies are likely to become a popular choice for health-conscious consumers seeking to improve their wellness in a simple and enjoyable way.\n\n"

#         "Context:\n"
#         "Sector: {sector}\n"
#         "Target Clients: {target_clients}\n"
#         "Resources: {resources}\n"
#         "Strategic Priorities: {strategic_priorities}\n"
#         "Project Name: {project_name}\n"
#         "Challenge Description: {challenge_description}\n"
#         "Purpose: {purpose}\n"
#         "Focus Constraints: {focus_constraints}\n\n"

#         "Keep in mind that who reads your output is already an expert on this sector, so make sure to use accurate, up-to-date sources, and come up with non-obvious results.\n"


#     ),
#     expected_output=(
#         "Produce a list of at least 10 trends with the following structure:\n\n"
#         "1. **Catchy Title**: Title that succinctly captures the essence of the trend.\n"
#         "2. **Body**: A detailed discussion including:\n"
#         "   - Description of the trend\n"
#         "   - Main drivers of the trend\n"
#         "   - Examples (mentioning relevant companies or products, where applicable)\n"
#         "   - Links to sources for further reference."
#     ),
#     tools=[exa_search_tool],  # Placeholder tool for conducting research
#     agent=researcher_agent
# )


In [ ]:
news_task = Task(
    description=(
        "Find ten recent news items from the sector mentioned below, that are relevant to the solution of the challenge "
        "or that could act as sources of inspiration.\n\n"

        "Context:\n"
        "Sector: {sector}\n"
        "Target Clients: {target_clients}\n"
        "Resources: {resources}\n"
        "Strategic Priorities: {strategic_priorities}\n"
        "Project Name: {project_name}\n"
        "Challenge Description: {challenge_description}\n"
        "Purpose: {purpose}\n"
        "Focus Constraints: {focus_constraints}\n\n"

    ),
    expected_output=(
        "A list of approximately 10 news items, each with:\n\n"
        "1. **Headline**: A catchy title summarizing the news item.\n"
        "2. **Summary**: A brief description of the news content.\n"
        "3. **Relevance**: A clear explanation of its connection to the challenge.\n"
        "4. **Source Link**: A clickable link to the source article."
    ),
    tools=[NewsSearchTool()],  # Placeholder tool for finding and analyzing recent news
    agent=researcher_agent
)


In [ ]:
customers_task = Task(
    description=(
        "For the topic and challenge below, search for all the information you can find about its target customers, clients, or users. "
        "Who are they? What are they like? Collect demographic, psychographic, and behavioral information from reputable sources. "
        "Based on your findings, attempt a segmentation of the market.\n\n"

        "Context:\n"
        "Sector: {sector}\n"
        "Target Clients: {target_clients}\n"
        "Resources: {resources}\n"
        "Strategic Priorities: {strategic_priorities}\n"
        "Project Name: {project_name}\n"
        "Challenge Description: {challenge_description}\n"
        "Purpose: {purpose}\n"
        "Focus Constraints: {focus_constraints}\n\n"

        "Expected output:\n"
        "Output a detailed analysis of the target users/customers/clients. For each, include:\n"
        "1. **Features of the Target Audience**: Describe the demographic, psychographic, and behavioral traits of the audience, including:\n"
        "   - Age, gender, income level, education, and location.\n"
        "   - Values, interests, lifestyle choices, and attitudes.\n"
        "   - Purchasing behaviors, product preferences, and engagement patterns.\n"
        "2. **Market Segmentation**: Provide a clear segmentation of the market based on the gathered information. Include segments with:\n"
        "   - Names or labels for the segments.\n"
        "   - Key characteristics of each segment.\n"
        "   - Potential relevance or appeal to the challenge.\n"
        "3. **Insights**: Highlight any weak signals or interesting patterns that could be leveraged for the challenge. \n"

        "Keep in mind that who reads your output is already an expert on this sector, so make sure to use accurate, up-to-date sources, and come up with non-obvious results.\n"

    ),
    expected_output=(
        "Provide the following deliverables:\n\n"
        "1. **Target Audience Features**: A detailed list of demographic, psychographic, and behavioral traits.\n"
        "2. **Market Segmentation**: A clear segmentation of the market, naming and describing each segment.\n"
        "3. **Relevance to Challenge**: A brief explanation of how these segments and audience traits relate to the challenge."
    ),
    tools=[search_tool, scrape_tool],  # Placeholder for tools to gather and analyze customer information
    agent=researcher_agent
)


In [ ]:
competitors_task = Task(
    description=(
        "Think step by step. Based on the challenge described below, what is the relevant market space or competitive field?"
        "Typically, this will be a product category within a certain geographic area."
        "Then, for the market space identified, search for the most important solutions (products or services) available from competitors. "
        "So make sure to indetify the right queries, search for existing competitors in that space, and list their solutions."
        "Keep in mind that in these markets there will typically be a mix of large, global players and smaller local ones."
        "Be extensive in your listing. \n\n"

        "Then, identify the key competitive factors for products currently on the market that the client might find valuable.\n\n"

        "Context:\n"
        "Sector: {sector}\n"
        "Target Clients: {target_clients}\n"
        "Resources: {resources}\n"
        "Strategic Priorities: {strategic_priorities}\n"
        "Project Name: {project_name}\n"
        "Challenge Description: {challenge_description}\n"
        "Purpose: {purpose}\n"
        "Focus Constraints: {focus_constraints}\n\n"

        "Expected output:\n"
        "1. **Competitor Solutions**: A detailed list of the most important products or services currently available in the market. For each solution, include:\n"
        "   - **Product/Service Name**: Name of the competitor solution.\n"
        "   - **Description**: A brief explanation of the product or service.\n"
        "   - **Provider**: Name of the company offering it.\n"
        "   - **Relevance**: How the solution relates to the challenge or target clients.\n"
        "   - **Link to Source**: URL to the product or service page, if available.\n\n"

        "2. **Competitive Factors**: A list of key competitive factors that define market success for similar solutions. For each factor, provide:\n"
        "   - **Factor Name**: Name or label for the competitive factor.\n"
        "   - **Description**: Explanation of why the factor is important.\n"
        "   - **Examples**: Instances of how competitors demonstrate these factors (if applicable).\n\n"

        "Ensure that the research is thorough and the output is actionable for addressing the challenge."
        "Keep in mind that who reads your output is already an expert on this sector, so make sure to use accurate, up-to-date sources, and come up with non-obvious results.\n"

    ),
    expected_output=(
        "Deliver the following:\n\n"
        "1. **Competitor Solutions**:\n"
        "   - A detailed list of relevant products or services from competitors, with descriptions, providers, relevance, and links to sources.\n\n"
        "2. **Competitive Factors**:\n"
        "   - A list of the key factors that make products successful in the market, with explanations and examples.\n"
    ),
    tools=[search_tool, scrape_tool],  # Placeholder for tools to conduct market research
    agent=researcher_agent
)


## Insights, opportunities, docs

In [ ]:
insight_task = Task(
    description=(
        "Generate five novel and thought-provoking insights based on the previous research findings, including the patents and research papers and insights from {doc_summary} if attached, else leave document part. "
        "An insight should uncover counterintuitive or surprising aspects about the current state of things, often by identifying underlying tensions "
        "or contradictions in the data. For example, there could be a disconnect between what consumers say they want and what is actually available, "
        "or conflicting trends that reveal hidden opportunities. Ensure that each insight is original and provides fresh perspectives.\n\n"

        "Example Insights:\n"
        "- There is a disconnect between the expert recommendation of intuitive eating and people's ingrained habits of following external food rules, "
        "relying on data rather than bodily cues.\n"
        "- There's a mismatch between people’s desire for natural, whole-food nutrition and the prevalence of artificial supplements in the performance nutrition market.\n\n"

        "Instructions:\n"
        "1. **Identify Tensions or Contradictions**: Look for contradictions or tensions in the research that might lead to surprising insights.\n"
        "2. **Generate Catchy Titles**: Each insight should have a compelling and catchy title that summarizes the tension or novel idea.\n"
        "3. **Provide Context**: After each insight, write a brief description that explains the underlying tension and the implications of the finding.\n"
        "4. **Cite Relevant Sources**: Reference the specific papers or patents from previous tasks that support or relate to each insight.\n"
        "5. **Explain Relevance to the Company**: For each insight, explain how it could impact the company and the challenge at hand, and why it is important for their strategic priorities.\n"
        "6. **Link to Challenge and Strategic Priorities**: Ensure that each insight is directly tied to the challenge, sector, target clients, and strategic priorities of the company."
    ),
    expected_output=(
        "A set of five novel, insightful, and cleverly formulated insights that address key contradictions or surprising trends in the research. "
        "Each insight should include a catchy title, a brief description, relevant sources from the research, and an explanation of how it applies to the specific company and its strategic priorities."
    ),
    tools=[],  # Placeholder tool for summarizing content
    context=[summarize_task],  # Link to the summary task for prior research context
    agent=insight_agent
)


In [ ]:
opportunities_task = Task(
    description=(
        "Here are a few examples of opportunity spaces that flow from insights, with their title and main ideas list:"

        "Insight1"
        "People don’t think of financial services as individuals, they think for their whole family. Especially during recessions, people rely more and more on their family network"
        "Opportunity space 1:"
        "Family bundling: Seeing the family as a wider financial unit, throughout each stage of life. Helping families maximize collective benefits; facilitating seamless household management; Creating meaningful experiences for younger users"
        "Ideas:"
        "Shared Family Dashboard"
        "Collective Minimal Spending: All family members’ spending counts towards monthly minimum & cash back points"
        "Payment Tracker: Keep track of upcoming utility payments and easily request family members to pay."
        "Family Goals: Set a real shared family goal (e.g. trip to Thailand) and make the process of saving fun and educational through gamified features."
        "Family Financial Education: Fun and interactive course for the whole family, including gamified weekly tasks."
        "Family Cards: Branded credit/debit cards with family initials or images."

        "Insight 2:"
        "People look for ways to improve their financial behavior, like saving plans and bargain hunting"
        "Opportunity space 2:"
        "Come as you are: we help people hack their daily routine, using behavioral design to nudge users for the optimum when life is up or down. Supporting clients as they develop healthy financial habits; Helping our clients navigate the low moments of life"
        "Ideas:"
        "Simulate-it: Personal simulation of loan-repayment program, credit limit, or investment forecasting."
        "A message from the future you: Plan ahead for financial security at an older age, taking into account predicted COL & playfully visualising your elderly appearance."
        "Round it up: Automatic round-up from every purchase towards a dedicated savings goal (e.g. university tuition)"
        "A piece of advice: Proactive notifications advising on ways to minimise fees and upgrade terms, based on personal usage data."
        "Build a Habit: Compete against yourself by setting personal goals and paying yourself when they’re accomplished."
        "Far from the eye: Increase your savings by automatically directing a percentage of your monthly salary to a dedicated savings zone."

        "Make sure the opportunity spaces are intriguingly formulated, that they are specific and non-obvious, but also that they encompass a wide range of ways to address the challenge. Make sure that the ideas, while innovative and interesting, stay true to the brand's DNA."

            "Context:\n"
            "Sector: {sector}\n"
            "Target Clients: {target_clients}\n"
            "Resources: {resources}\n"
            "Strategic Priorities: {strategic_priorities}\n"
            "Project Name: {project_name}\n"
            "Challenge Description: {challenge_description}\n"
            "Purpose: {purpose}\n"
            "Focus Constraints: {focus_constraints}"
    ),
    expected_output=(
        "Five opportunity spaces, each composed of a catchy title, a short description, a reference to the insights that support it, and a list of five relevant product or service ideas."
    ),
    tools = [],
    context = [insight_task],
    agent = opportunities_strategist

)

In [ ]:
opp_image_task = Task(
    description=(
        "For each of the opportunity spaces present in {opportunity_spaces}, pick the first idea mentioned and **generate** a simple, realistic, tasteful image of the product described. Make the image simple, elegant and essential."
        "Make sure the image stays true to the brand's DNA."

        "Context:\n"
        "Sector: {sector}\n"
        "Target Clients: {target_clients}\n"
        "Resources: {resources}\n"
        "Strategic Priorities: {strategic_priorities}\n"
        "Project Name: {project_name}\n"
        "Challenge Description: {challenge_description}\n"
        "Purpose: {purpose}\n"
        "Focus Constraints: {focus_constraints}"

    ),
    expected_output=(
        "Genrate A set of five images, one for each opportunity space"
    ),
    tools = [],
    context = [opportunities_task],
    agent = opp_images
)

In [ ]:
doc_analyze_task = Task(
    description=(
        "Analyze the attached documents {documents} to identify insights that could be relevant to finding solutions to the challenge below.  Make sure to include direct quotes from the text and references when relevant."
        "Output a summary of the content of the document, with particular focus on the elements that could help solve the challenge. If documents {documents} are not attached you should move on to other task. Answer should be in english"

            "Context:\n"
            "Sector: {sector}\n"
            "Target Clients: {target_clients}\n"
            "Resources: {resources}\n"
            "Strategic Priorities: {strategic_priorities}\n"
            "Project Name: {project_name}\n"
            "Challenge Description: {challenge_description}\n"
            "Purpose: {purpose}\n"
            "Focus Constraints: {focus_constraints}"
      ),
    expected_output=(
        "Output a summary of the content of the document, with particular focus on the elements that could help solve the challenge."
        "If documents {documents} are not attached you should move on."
    ),
    tools = [],
    agent=doc_analyst
)


# CREWS

In [ ]:
#Patents crew

patents_crew = Crew(
    agents=[
        strategist_agent,
        researcher_agent,
        #scraper_agent,
        writer_agent,

      ],

    tasks=[
        generate_queries_task,
        patent_search_task,
        #patent_search_task,
        #scrape_content_task,
        select_patents_task,
        summarize_task,

    ],

    #process=Process.hierarchical,
    #manager_agent=manager,
    #manager_llm=manager_llm,

    process=Process.sequential,
    #planning=True,
    verbose=True,
	  memory=False,
    #cache=False,
    #share_crew=False,
    #output_log_file="outputs/content_plan_log.txt",
    #max_rpm=50,
    #output_name='patents_output'

)

In [ ]:
#Scholar crew

scholar_crew = Crew(
    agents=[
        strategist_agent,
        researcher_agent,
        #scraper_agent,
        writer_agent,
      ],

    tasks=[
        generate_queries_task,
        scholar_search_task,
        #patent_search_task,
        #scrape_content_task,
        select_papers_task,
        summarize_task,

    ],

    #process=Process.hierarchical,
    #manager_agent=manager,
    #manager_llm=manager_llm,

    process=Process.sequential,
    #planning=True,
    verbose=True,
	  memory=False,
    #cache=False,
    #share_crew=False,
    #output_log_file="outputs/content_plan_log.txt",
    #max_rpm=50,
    output_name='scholar_output'

)

In [ ]:
#Trends crew

trends_crew = Crew(
    agents=[
        researcher_agent,
      ],

    tasks=[
        trends_task,

    ],

    #process=Process.hierarchical,
    #manager_agent=manager,
    #manager_llm=manager_llm,

    process=Process.sequential,
    #planning=True,
    verbose=True,
	  memory=False,
    #cache=False,
    #share_crew=False,
    #output_log_file="outputs/content_plan_log.txt",
    #max_rpm=50,
    output_name='trends_output'

)

In [ ]:
#News crew

news_crew = Crew(
    agents=[
        researcher_agent,
      ],

    tasks=[
        news_task,

    ],

    #process=Process.hierarchical,
    #manager_agent=manager,
    #manager_llm=manager_llm,

    process=Process.sequential,
    #planning=True,
    verbose=True,
	  memory=False,
    #cache=False,
    #share_crew=False,
    #output_log_file="outputs/content_plan_log.txt",
    #max_rpm=50,
    output_name='news_output'

)

In [ ]:
#customers crew

customers_crew = Crew(
    agents=[
        researcher_agent,
      ],

    tasks=[
         customers_task,

    ],

    #process=Process.hierarchical,
    #manager_agent=manager,
    #manager_llm=manager_llm,

    process=Process.sequential,
    #planning=True,
    verbose=True,
	  memory=False,
    #cache=False,
    #share_crew=False,
    #output_log_file="outputs/content_plan_log.txt",
    #max_rpm=50,
    output_name='customers_output'

)

In [ ]:
#competitors crew

competitors_crew = Crew(
    agents=[
        researcher_agent,
      ],

    tasks=[
        competitors_task,

    ],

    #process=Process.hierarchical,
    #manager_agent=manager,
    #manager_llm=manager_llm,

    process=Process.sequential,
    #planning=True,
    verbose=True,
	  memory=False,
    #cache=False,
    #share_crew=False,
    #output_log_file="outputs/content_plan_log.txt",
    #max_rpm=50,
    output_name='competitors_output'

)

In [ ]:
#Insights crew

insights_crew = Crew(
    agents=[
        # doc_analyst,
        insight_agent,
      ],

    tasks=[
        # doc_analyze_task,
        insight_task,

    ],

    #process=Process.hierarchical,
    #manager_agent=manager,
    #manager_llm=manager_llm,

    process=Process.sequential,
    #planning=True,
    verbose=True,
	  memory=False,
    #cache=False,
    #share_crew=False,
    #output_log_file="outputs/content_plan_log.txt",
    #max_rpm=50,
    output_name='insights'

)

## New Crew

In [ ]:
# Opportunity Space

opp_spaces_crew = Crew(
    agents=[
        opportunities_strategist,
      ],

    tasks=[
        opportunities_task,


    ],

    #process=Process.hierarchical,
    #manager_agent=manager,
    #manager_llm=manager_llm,

    process=Process.sequential,
    #planning=True,
    verbose=True,
	  memory=False,
    #cache=False,
    #share_crew=False,
    #output_log_file="outputs/content_plan_log.txt",
    #max_rpm=50,
    output_name='Opportunity Spaces'

)

In [ ]:
opp_images_crew = Crew(
    agents=[
        opp_images,
      ],

    tasks=[
        opp_image_task,
    ],

    #process=Process.hierarchical,
    #manager_agent=manager,
    #manager_llm=manager_llm,

    process=Process.sequential,
    #planning=True,
    verbose=True,
	  memory=False,
    #cache=False,
    #share_crew=False,
    #output_log_file="outputs/content_plan_log.txt",
    #max_rpm=50,
    output_name='Opportunity Space Images'

)

In [ ]:
doc_analyst_crew = Crew(
    agents=[
        doc_analyst,
      ],

    tasks=[
        doc_analyze_task,
    ],

    #process=Process.hierarchical,
    #manager_agent=manager,
    #manager_llm=manager_llm,

    process=Process.sequential,
    #planning=True,
    verbose=True,
	  memory=False,
    #cache=False,
    #share_crew=False,
    #output_log_file="outputs/content_plan_log.txt",
    #max_rpm=50,
    output_name='Opportunity Space Images'

)

# Inputs

In [ ]:
input_1 = {
    # Company inputs
    "company_name": "Coffee Bean Technologies",
    "sector": "Coffee Manufacturing",
    "target_clients": "HoReCa and brands who sell coffee",
    "resources": "Coffee plantations, machinery for harvesting, toasting, brewing, packaging; distribution network",
    "strategic_priorities": "Identifying new products to penetrate existing and new markets, particularly companies targeting younger consumers",

    # Project inputs
    "project_name": "Coffee Bean Coating Technologies",
    "challenge_description": "Identifying technologies that enable coating coffee beans with a thin layer containing color and nutrients",
    "purpose": "Enriching cofee beans by coating them with colors and nutrients ",
    "focus_constraints": " ",

}

input_2 = {
    # Company inputs
    "company_name": "Campari Group",
    "sector": "Beverages / Alcoholic Spirits",
    "target_clients": "Young consumers seeking trendy and flavorful drink options",
    "resources": "Strong brand legacy, high-quality ingredients, advanced distillation techniques, and a portfolio of iconic spirits",
    "strategic_priorities": "Appeal to a younger demographic; innovate while preserving brand heritage; enhance brand recognition and loyalty; capitalize on social media and influencer marketing",

    # Project inputs
    "project_name": "Development of a Modernized Campari Drink",
    "challenge_description": "Create a new version of the traditional Campari drink that aligns with current trends, appeals to younger consumers, and retains the brand's legacy of excellence in flavor and craftsmanship",
    "purpose": "Attract and engage young consumers while honoring the iconic nature of Campari, ensuring relevance in a competitive and evolving market",
    "focus_constraints": "Maintain Campari's signature flavor profile while incorporating modern elements that resonate with younger audiences, such as unique ingredients, vibrant presentation, and sustainable packaging",
}




input_3 = {
    # Document
    "document" : """Partial product list
1. Cones:

Amando Cono Vaniglia, Cacao e Nocciola: Vanilla-flavored gelato with a cocoa and hazelnut ripple, topped with crunchy Sicilian almond bits, all encased in a gluten-free cone.
SAM MONTANA
2. Tubs:

Amando Tub Vanilla, Chocolate & Hazelnut: A trio of classic flavors—vanilla, chocolate, and hazelnut—crafted from almond milk, providing a creamy and indulgent experience.
SAM MONTAÑA MALTA

Amando Tub Vanilla & Salted Caramel: Smooth vanilla gelato swirled with salted caramel, offering a balance of sweet and savory notes.
SAM MONTAÑA MALTA

3. Stick Bars:

Stecco Amando Cacao e Lampone: Vanilla-flavored gelato with a raspberry ripple, coated in cacao and sprinkled with crunchy Sicilian almond bits.
SAM MONTANA

Stecco Amando Frutta: Vanilla-flavored gelato with three different fruit coatings: peach, red berries, and apple & kiwi, providing a refreshing and fruity twist.
SAM MONTANA

Amando Stecco Pesca e Frutti Rossi: Vanilla-flavored gelato with dual coatings of peach and red berries, offering a delightful fruit-infused treat.
SAM MONTANA

4. Sandwiches:

Biscotto Amando con Frutti di Bosco: Vanilla gelato made from almond milk, rippled with red fruits, and sandwiched between two gluten-free biscuits made from cereals with cranberries.


Online reviews
Position        Review Title        Reviewer Info        Reviewer Name        Review Text
1        Gelato all'italiana        opinione inserita da Jimorris il 13/02/2020        Jimorris        "Il gelato è una specialità italiana e Sammontana è uno dei nomi più importanti e noti di questo settore. Lo conosco fin da bambino con il suo prodotto storico: il barattolino; tuttavia adesso l'azienda è cresciuta e offre una gamma di nuovi prodotti con la linea tre stelle. Io sono felice quando vedo i prodotti di quest'azienda perchè sono sicuro che sono prodotti sicuri e di qualità, l'ideali per la mia bambina. Ricordo che in uno dei parchi che frequentiamo ci sono queste linee di prodotti e fra questi mia figlia adora la coppetta bianco e nero dei PjMask. Noi abbiamo preso gelati stecco davvero ottimi. Quest'estate ho avuto il piacere di scoprire la loro linea di gelati completamente vegetali, ottimi di sapore e senza alcun derivato animale. Tipologia di prodotti che preferisco e anche mia moglie perchè meno ingrassanti dei classici gelati, ma ho avuto paicere di notare che sono buoni come quelli a base di latte.
Segnala contenuto"
2        Gelato        opinione inserita da Mickycri il 25/01/2018        Mickycri        "Il gelato è tra i dolci più buoni che ci sono e Sammontana è sicuramente una garanzia di grande qualità e gusto. Il suo assortimento è veramente vasta, dai diversi gusti delle vaschette che si possono trovare in diversi formati, ai gelati da passeggio che si trovano anche al supermercato in comodiss...
Segnala contenuto"
3        Il gelato all'italiana        opinione inserita da Supermario64bros il 02/10/2017        Supermario64bros        "La Sammontana è un'azienda specializzata nella produzione di gelati. Noi italiani siamo bravi a fare il gelato e la Sammontana è tra i più bravi. Produce una gran varietà di gelati, ghiaccioli, biscotti, coni e stecchi, e sicuramente quello che tutti noi più conosciamo ed amiamo è il Barattolino Sam...
Segnala contenuto"
4        Il gelato italiano        opinione inserita da Anonimo il 26/09/2017        Anonimo        "La marca Sammontana è un ottima scelta tra i gelati italiani confezionati e mi ricordo che li prendevo sempre perchè erano tra quelli più convenienti nel rapporto quantità-qualità-prezzo. Ora invece i prezzi li stanno un po' aumentando sopratutto nei bar, però rimane ancora la qualità del prodotto c...
Segnala contenuto"
5        Sammontana        opinione inserita da Xroswars il 15/08/2017        Xroswars        "Sammontana è una delle marche che preferisco di gelato, leggermente più costosa di altre ma sicuramente un gelato buono e di alta qualità. Personalmente il gelato sammontana è a mio avviso uno dei migliori che disponiamo in Italia e sono prodotti così tanti tipi di gelati diversi che secondo me diff...
Segnala contenuto"
6        Sammontana        opinione inserita da Francesca FiFi Bruno il 31/05/2017        Francesca FiFi Bruno        "Se penso ai gelati confezionati la marca Sammontana è una delle prime che mi passa per la mente. Mi ricordo benissimo la pubblicità tormentone che ha dominato la mia infanzia, soprattutto ricordo il simbolo: un gelatino che si lecca la boccuccia. La Sammontana è un marca di gelati tutta italiana che...
Segnala contenuto"
7        la sammontana        opinione inserita da Anonimo il 31/05/2017        Anonimo        "La marca Sammontana è una delle migliori se non la migliore marca in assoluto di gelati completamente italiani infatti anche la Algida ci mette del suo (e poi altre) ma questa Sammontana ha tradizione e sentimento in quello che fa anche se rimane in ogni caso una fabbrica con produzione a livello in...
Segnala contenuto"
8        Sammontana        opinione inserita da Zoe79 il 31/05/2017        Zoe79        "Sammontana, leader nella produzioni di gelati, lo reputo un marchio da associare appunto all' estate, data la popolarità dei suoi prodotti, alcuni dei quali dei veri e propri classici della gelateria. A tal riguardo segnalo i numerosi barattolini, realizzati nei più svariati gusti ed ideali da consu...
Segnala contenuto"
9        sammontana        opinione inserita da Anonimo il 15/04/2017        Anonimo        "Sammontana è un marchio di prodotti a base di gelati e dolci surgelati di vario genere. E' uno dei marchi di gelati che acquisto spesso. Si tratta di prodotti di ottima qualità. Per quanto riguarda il gusto hanno un buon sapore abbastanza intenso. Per quanto riguarda la consistenza si tratta di gela...
Segnala contenuto"
10        Gelati all'italiana        opinione inserita da Galaxy93 il 11/04/2017        Galaxy93        "La scelta di gelati della Sammontana è davvero molto vasta e quindi basterà sceglier quello che più vi convince ed acquistarlo anche perché i prezzi sono sempre a favore del cliente. Ho assaggiato il barattolino Sammontana e me ne sono letteralmente innamorato dopo il primo cucchiaio. Questa marca è...
Segnala contenuto"
11        Gelati all' Italiana molto buoni        opinione inserita da Marziano il 10/04/2017        Marziano        "La Sammontana secondo me è un azienda che propone prodotti di buona o alta qualità, forse un pò sottovalutata rispetto ai colossi del settore, Algida e Motta, ma che in molti casi non ha nulla da invidiare. I gelati hanno dei gusti molto buoni, tutti quelli che mi è capitato di assaggiare, sia in va...
Segnala contenuto"
12        Tra i migliori gelati in assoluto        opinione inserita da Alereviw il 05/02/2017        Alereviw        "Sammontana come dice il logo è uno dei brand più famosi in Italia per la gelateria confezionata. Si possono trovare moltissimi prodotti nei vari supermercati e sempre di ottima qualità. Personalmente fin da piccolo apprezzo molto i coni gelato e i gelati in scatola, in particolare ai gusti panna e a...
Segnala contenuto"
13        Sammontana        opinione inserita da Alep il 23/01/2017        Alep        "Il marchio Sammontana è davvero famoso e per me indice di qualità sin da quando sono bambina. Chiaramente i loro prodotti sono famosi soprattutto per il Bimbi proprio perché riguardano i gelati, una mia grande passione. Devo dire che nel corso degli anni hanno introdotto tantissimi prodotti ma i cla...
Segnala contenuto"
14        Sammontana        opinione inserita da Suckitandsee17 il 20/12/2016        Suckitandsee17        "La Sammontana è una delle marche di gelati più famose in Italia, e che a mio avviso domina assolutamente la concorrenza, forse l'unica a fare concorrenza è l'Algida. Ha lanciato il classico barattolino che oramai è storia in Italia, ma anche tanti altri tipi di gelato e detiene assolutamente uno dei...
Segnala contenuto"
15        La marca di gelati Sammontana        opinione inserita da Vitocatozzo il 20/12/2016        Vitocatozzo        "Marca di gelati all'italiana affermata ormai da diversi anni. La sede centrale di questa azienda che produce dei gelati di sicura qualità è ad Empoli, a circa trenta chilometri da Firenze. Questo spiega il motivo per cui per diverso tempo il marchio Sammontana è stato anche lo sponsor della locale s...
Segnala contenuto"
16        Sammontana        opinione inserita da Supermitica il 06/12/2016        Supermitica        "Con l'arrivo dell'estate generalmente mi capita sempre di vedere contemporaneamente anche la comparsa delle prime pubblicità Sammontana. Questa è una delle marche di gelato penso più rifornita in quanto non si occupa soltanto di gelati venduti a pezzi singoli, per farci intendere quelli dei bar, ma ...
Segnala contenuto"
17        Sammontana        opinione inserita da Lucias il 24/10/2016        Lucias        "Se si parla di gelati, mi viene in mente subito la Sammontana. La Sammontana è una azienda italiana, la più famosa sicuramente che produce gelato. I gelati Sammontana fanno parte della mia vita, compravo i barattolini d'estate e ne facevo grandi scorte, quando finiva il gelato, mia figlia lo usava p...
Segnala contenuto"
18        Tanti gusti nel barattolino Sammontana        opinione inserita da Rossella1978 il 17/10/2016        Rossella1978        "La Sammonana offre gelati industriali di buona qualità. A volte quanto si dice "gelato industriale" qualcuno sembra storcere il naso ma io penso all'etichetta e al fatto che in qualche modo siano più controllati rispetto a quelli artigianali. Ovviamente come ingredienti quello artigianale sarà senza...
Segnala contenuto"
19        Una meraviglia        opinione inserita da Marysyna il 02/10/2016        Marysyna        "Conosco questa nota marca da vari anni. Di solito acquisto i deliziosi gelati dal bar, a volte anche dal supermercato. Buonissime sia le numerose varietà sfuse che in formato famiglia. Gustosi i cornetti, biscotti e le coppette. Altrettanto posso dire dei barattoli e delle sorbettiere. I gusti sempr...
Segnala contenuto"
20        Marca grandiosa        opinione inserita da Briciola il 20/09/2016        Briciola        "Sammontana è un marchio grandioso nel campo dei gelati, in Italia è un'instituzione e quasi ogni bar vende i gelati brandizzati proprio Sammontana. Io, specie nel periodo estivo, compro il Barattolino che è composto da due gusti. Di solito prendo crema e pistacchio perchè credo che la Sammontana fac...
Segnala contenuto"
21        La mia estate italiana!        opinione inserita da Margherita123 il 08/09/2016        Margherita123        "Bella la pubblicità della Sammontana. Tutta l'Italia unita da un semplice barattolino di gelato, ognuno con la propria serata estiva da trascorrere in balcone, davanti la tv, con le amiche ma sempre e comunque con un buon gelato. Ovviamente Sammontana. Effettivamente di questa marca mi piacciono mol...
Segnala contenuto"
22        W il gelato        opinione inserita da Melinda7311 il 19/08/2016        Melinda7311        "Quando si parla di gelati , la mia mente va subito ai gelati a marchio Sammontana. La Sammontana mi ricorda la mia infanzia quando mia madre comprava il barattolino di gelato, era un must a casa nostra non mancava mai. Da adulta lo compro anch'io, e mia madre ancora lo fa. Il gelato è ottimo, nel te...
Segnala contenuto""",
    # Company inputs
    "company_name": "Sammontana",
    "sector": "Food & Beverage / Consumer Packaged Goods",
    "target_clients": "Italian consumers seeking high-quality gelato and bakery products",
    "resources": "Established reputation; advanced production facilities; strong distribution network",
    "strategic_priorities": "Expand product lines to include health-conscious options; enhance brand presence internationally; invest in sustainable production practices",

    # Project inputs
    "project_name": "Development of Plant-Based and Low-Sugar Gelato Options",
    "challenge_description": "Develop plant-based and low-sugar gelato options to cater to health-conscious consumers in Italy",
    "purpose": "Maintain authentic taste and texture while using alternative ingredients",
    "focus_constraints": "Ensure new products meet quality standards and appeal to traditional gelato consumers",
}



input_4 = {
    # Company inputs
    "company_name": "Campari Group",
    "sector": "Beverages / Alcoholic Spirits",
    "target_clients": "Young consumers seeking trendy and flavorful drink options",
    "resources": "Strong brand legacy, high-quality ingredients, advanced distillation techniques, and a portfolio of iconic spirits",
    "strategic_priorities": "Appeal to a younger demographic; innovate while preserving brand heritage; enhance brand recognition and loyalty; capitalize on social media and influencer marketing",

    # Project inputs
    "project_name": "Development of a Modernized Campari Drink",
    "challenge_description": "Create a new version of the traditional Campari drink that aligns with current trends, appeals to younger consumers, and retains the brand's legacy of excellence in flavor and craftsmanship",
    "purpose": "Attract and engage young consumers while honoring the iconic nature of Campari, ensuring relevance in a competitive and evolving market",
    "focus_constraints": "Maintain Campari's signature flavor profile while incorporating modern elements that resonate with younger audiences, such as unique ingredients, vibrant presentation, and sustainable packaging",
}




In [ ]:
#Input swicher

my_input=input_4

In [ ]:
# Test news crew

news_result = news_crew.kickoff(inputs=my_input)

In [ ]:
## Test trends crew

trends_result = trends_crew.kickoff(inputs=my_input)

In [ ]:
## Test customers crew

customers_result = customers_crew.kickoff(inputs=my_input)

In [ ]:
## Test competitors crew

competitors_result = competitors_crew.kickoff(inputs=my_input)

# FLOWS

In [ ]:
from crewai.flow.flow import Flow, start, listen
from crewai import Flow
from crewai.flow.flow import listen,start

In [ ]:
#Flows with parametrized inputs
#RESEARCH FLOWS


class InsightGen(Flow):
    @start()
    async def start_parallel_execution(self):
        # This is the starting point of the flow. The @start() decorator indicates
        # that this method is run when the flow begins.

        # Kick off two crews (patents_crew and scholar_crew) asynchronously in parallel.
        # 'inputs=input_3' means both crews receive 'input_3' as their input parameters.
        patents_future = patents_crew.kickoff_async(inputs=my_input)
        scholar_future = scholar_crew.kickoff_async(inputs=my_input)
        trends_future = trends_crew.kickoff_async(inputs=my_input)
        news_future = news_crew.kickoff_async(inputs=my_input)
        customers_future = customers_crew.kickoff_async(inputs=my_input)
        competitors_future = competitors_crew.kickoff_async(inputs=my_input)

        # patents_future = asyncio.create_task(self.run_crew_for_task(self.patents_crew, self.patent_search_task))
        # scholar_future = asyncio.create_task(self.run_crew_for_task(self.scholar_crew, self.scholar_search_task))
        # trends_future = asyncio.create_task(self.run_crew_for_task(self.trends_crew, self.trends_task))

        # asyncio.gather runs both futures concurrently and waits until both complete.
        patents_output, scholar_output, trends_output, news_output, customers_output, competitors_output = await asyncio.gather(patents_future, scholar_future, trends_future, news_future, customers_future, competitors_future) # trends_future was missing

        # One the runs have completed, extract raw outputs if they exist; otherwise, convert to string.
        # This ensures we have a consistent way of handling the crew outputs.
        patents_data = patents_output.raw_output if hasattr(patents_output, 'raw_output') else str(patents_output)
        scholar_data = scholar_output.raw_output if hasattr(scholar_output, 'raw_output') else str(scholar_output)
        trends_data = trends_output.raw_output if hasattr(trends_output, 'raw_output') else str(trends_output)
        news_data = news_output.raw_output if hasattr(news_output, 'raw_output') else str(news_output)
        customers_data = customers_output.raw_output if hasattr(customers_output, 'raw_output') else str(customers_output)
        competitors_data = competitors_output.raw_output if hasattr(competitors_output, 'raw_output') else str(competitors_output)

        # Store results in the flow’s shared state for subsequent tasks to use.
        self.state["patents_crew_results"] = patents_data
        self.state["scholar_crew_results"] = scholar_data
        self.state["trends_crew_results"] = trends_data
        self.state["news_crew_results"] = news_data
        self.state["customers_crew_results"] = customers_data
        self.state["competitors_crew_results"] = competitors_data

        # Return the combined results so that the next tasks in the flow can access them.
        return {"patents": patents_data, "scholar": scholar_data, "trends": trends_data, "news": news_data, "customers": customers_data, "competitors": competitors_data}


#####################################################################################


#DOC ANALYSIS
    @listen(start_parallel_execution)
    def check_and_trigger_doc_analyst(self, parallel_results):
        # This method runs after 'start_parallel_execution' completes.
        # It checks if there is a 'document' field in input_3 and if it is not empty.
        # If a document is provided, it triggers the doc_analyst_crew.

        if "document" in my_input and my_input["document"]:
            # Prepare the input for the doc_analyst_crew using fields from input_3.
            doc_analyst_input = {
                "documents": my_input["document"],
                "sector": my_input["sector"],
                "target_clients": my_input["target_clients"],
                "resources": my_input["resources"],
                "strategic_priorities": my_input["strategic_priorities"],
                "project_name": my_input["project_name"],
                "challenge_description": my_input["challenge_description"],
                "purpose": my_input["purpose"],
                "focus_constraints": my_input["focus_constraints"]
            }

            # Run the doc_analyst crew synchronously and retrieve its output.
            doc_analyst_output = doc_analyst_crew.kickoff(doc_analyst_input)
            doc_analyst_data = doc_analyst_output.raw_output if hasattr(doc_analyst_output, "raw_output") else str(doc_analyst_output)

            # Store the document analysis results in the state.
            self.state["doc_analyst_results"] = doc_analyst_data

            # Return the previous parallel results combined with the doc_analyst data.
            return {**parallel_results, "doc_analyst": doc_analyst_data}
        else:
            # If no document was provided, just return the parallel results from before.
            return parallel_results


#####################################################################################


    #INSIGHT GENERATION
    @listen(check_and_trigger_doc_analyst)
    def generate_insights(self, previous_results):
        # This method runs after the doc_analyst check is done.
        # It aggregates data from patents, scholar, and doc_analyst (if available),
        # along with input_3 fields, and feeds them into the insights_crew.

        combined_output = {
            "patents": self.state.get("patents_crew_results", ""),
            "scholar": self.state.get("scholar_crew_results", ""),
            "trends": self.state.get("trends_crew_results", ""),
            "news": self.state.get("news_crew_results", ""),
            "customers": self.state.get("customers_crew_results", ""),
            "competitors": self.state.get("competitors_crew_results", ""),
            "doc_summary": self.state.get("doc_analyst_results", ""),  # may be empty if no doc was given
            "sector": my_input["sector"],
            "target_clients": my_input["target_clients"],
            "resources": my_input["resources"],
            "strategic_priorities": my_input["strategic_priorities"],
            "project_name": my_input["project_name"],
            "challenge_description": my_input["challenge_description"],
            "purpose": my_input["purpose"],
            "focus_constraints": my_input["focus_constraints"],
        }

        # Run the insights_crew with the combined data.
        insights_output = insights_crew.kickoff(combined_output)
        insights_data = insights_output.raw_output if hasattr(insights_output, "raw_output") else str(insights_output)

        # Save the insights results to the state.
        self.state["insights_crews_results"] = insights_data

        # Return insights data for the next task.
        return insights_data


#####################################################################################


  #OPPORTUNITY SPACES GENERATION
    @listen(generate_insights)
    def generate_opportunity_spaces(self, insights_data):
        # This method listens to the output of generate_insights.
        # It uses the insights data and other input parameters to run the opp_spaces_crew.

        opp_spaces_input = {
            "insights": insights_data,
            "sector": my_input["sector"],
            "target_clients": my_input["target_clients"],
            "resources": my_input["resources"],
            "strategic_priorities": my_input["strategic_priorities"],
            "project_name": my_input["project_name"],
            "challenge_description": my_input["challenge_description"],
            "purpose": my_input["purpose"],
            "focus_constraints": my_input["focus_constraints"]
        }

        # Run the opportunity spaces crew.
        opp_spaces_output = opp_spaces_crew.kickoff(opp_spaces_input)
        opp_spaces_data = opp_spaces_output.raw_output if hasattr(opp_spaces_output, 'raw_output') else str(opp_spaces_output)

        # Store the results in the state.
        self.state["opp_spaces_results"] = opp_spaces_data

        # Return the opportunity spaces data for the next step.
        return opp_spaces_data


#####################################################################################



#OPPORTUNITY IMAGES GENERATION
    @listen(generate_opportunity_spaces)
    def generate_opportunity_images(self, opp_spaces_data):
        # This method runs after the opportunity spaces are generated.
        # It uses the opp_spaces_data and other context fields to run the opp_images_crew.

        opp_images_input = {
            "opportunity_spaces": opp_spaces_data,
            "sector": my_input["sector"],
            "target_clients": my_input["target_clients"],
            "resources": my_input["resources"],
            "strategic_priorities": my_input["strategic_priorities"],
            "project_name": my_input["project_name"],
            "challenge_description": my_input["challenge_description"],
            "purpose": my_input["purpose"],
            "focus_constraints": my_input["focus_constraints"]
        }

        # Run the opportunity images crew.
        opp_images_output = opp_images_crew.kickoff(opp_images_input)
        opp_images_data = opp_images_output.raw_output if hasattr(opp_images_output, 'raw_output') else str(opp_images_output)

        # Store final opportunity images results.
        self.state["opp_images_results"] = opp_images_data

        # Return this final piece of output (images).
        return opp_images_data

    async def kickoff_async(self):
        # This method overrides the default asynchronous kickoff of the Flow.
        # It runs the entire flow asynchronously and returns a dictionary of final results.

        result = await super().kickoff_async()
        return {
            "patents_data": self.state.get("patents_crew_results"),
            "scholar_data": self.state.get("scholar_crew_results"),
            "trends_data":self.state.get("trends_crew_results"),
            "insights_data": self.state.get("insights_crews_results"),
            "opp_spaces_data": self.state.get("opp_spaces_results"),
            "opp_images_data": self.state.get("opp_images_results"),
            "news_data": self.state.get("news_crew_results"),
            "customers_data": self.state.get("customers_crew_results"),
            "competitors_data": self.state.get("competitors_crew_results"),
        }





In [ ]:
# #Flows with parametrized inputs - OLD



# class InsightGen(Flow):
#     @start()
#     async def start_parallel_execution(self):
#         # This is the starting point of the flow. The @start() decorator indicates
#         # that this method is run when the flow begins.

#         # Kick off two crews (patents_crew and scholar_crew) asynchronously in parallel.
#         # 'inputs=input_3' means both crews receive 'input_3' as their input parameters.
#         patents_future = patents_crew.kickoff_async(inputs=my_input)
#         scholar_future = scholar_crew.kickoff_async(inputs=my_input)

#         # asyncio.gather runs both futures concurrently and waits until both complete.
#         patents_output, scholar_output = await asyncio.gather(patents_future, scholar_future)

#         # One the runs have completed, extract raw outputs if they exist; otherwise, convert to string.
#         # This ensures we have a consistent way of handling the crew outputs.
#         patents_data = patents_output.raw_output if hasattr(patents_output, 'raw_output') else str(patents_output)
#         scholar_data = scholar_output.raw_output if hasattr(scholar_output, 'raw_output') else str(scholar_output)

#         # Store results in the flow’s shared state for subsequent tasks to use.
#         self.state["patents_crew_results"] = patents_data
#         self.state["scholar_crew_results"] = scholar_data

#         # Return the combined results so that the next tasks in the flow can access them.
#         return {"patents": patents_data, "scholar": scholar_data}

#     @listen(start_parallel_execution)
#     def check_and_trigger_doc_analyst(self, parallel_results):
#         # This method runs after 'start_parallel_execution' completes.
#         # It checks if there is a 'document' field in input_3 and if it is not empty.
#         # If a document is provided, it triggers the doc_analyst_crew.

#         if "document" in my_input and my_input["document"]:
#             # Prepare the input for the doc_analyst_crew using fields from input_3.
#             doc_analyst_input = {
#                 "documents": my_input["document"],
#                 "sector": my_input["sector"],
#                 "target_clients": my_input["target_clients"],
#                 "resources": my_input["resources"],
#                 "strategic_priorities": my_input["strategic_priorities"],
#                 "project_name": my_input["project_name"],
#                 "challenge_description": my_input["challenge_description"],
#                 "purpose": my_input["purpose"],
#                 "focus_constraints": my_input["focus_constraints"]
#             }

#             # Run the doc_analyst crew synchronously and retrieve its output.
#             doc_analyst_output = doc_analyst_crew.kickoff(doc_analyst_input)
#             doc_analyst_data = doc_analyst_output.raw_output if hasattr(doc_analyst_output, "raw_output") else str(doc_analyst_output)

#             # Store the document analysis results in the state.
#             self.state["doc_analyst_results"] = doc_analyst_data

#             # Return the previous parallel results combined with the doc_analyst data.
#             return {**parallel_results, "doc_analyst": doc_analyst_data}
#         else:
#             # If no document was provided, just return the parallel results from before.
#             return parallel_results


# #INSIGHT GENERATION
#     @listen(check_and_trigger_doc_analyst)
#     def generate_insights(self, previous_results):
#         # This method runs after the doc_analyst check is done.
#         # It aggregates data from patents, scholar, and doc_analyst (if available),
#         # along with input_3 fields, and feeds them into the insights_crew.

#         combined_output = {
#             "patents": self.state.get("patents_crew_results", ""),
#             "scholar": self.state.get("scholar_crew_results", ""),
#             "doc_summary": self.state.get("doc_analyst_results", ""),  # may be empty if no doc was given
#             "sector": my_input["sector"],
#             "target_clients": my_input["target_clients"],
#             "resources": my_input["resources"],
#             "strategic_priorities": my_input["strategic_priorities"],
#             "project_name": my_input["project_name"],
#             "challenge_description": my_input["challenge_description"],
#             "purpose": my_input["purpose"],
#             "focus_constraints": my_input["focus_constraints"],
#         }

#         # Run the insights_crew with the combined data.
#         insights_output = insights_crew.kickoff(combined_output)
#         insights_data = insights_output.raw_output if hasattr(insights_output, "raw_output") else str(insights_output)

#         # Save the insights results to the state.
#         self.state["insights_crews_results"] = insights_data

#         # Return insights data for the next task.
#         return insights_data


# #OPPORTUNITY SPACES GENERATION
#     @listen(generate_insights)
#     def generate_opportunity_spaces(self, insights_data):
#         # This method listens to the output of generate_insights.
#         # It uses the insights data and other input parameters to run the opp_spaces_crew.

#         opp_spaces_input = {
#             "insights": insights_data,
#             "sector": my_input["sector"],
#             "target_clients": my_input["target_clients"],
#             "resources": my_input["resources"],
#             "strategic_priorities": my_input["strategic_priorities"],
#             "project_name": my_input["project_name"],
#             "challenge_description": my_input["challenge_description"],
#             "purpose": my_input["purpose"],
#             "focus_constraints": my_input["focus_constraints"]
#         }

#         # Run the opportunity spaces crew.
#         opp_spaces_output = opp_spaces_crew.kickoff(opp_spaces_input)
#         opp_spaces_data = opp_spaces_output.raw_output if hasattr(opp_spaces_output, 'raw_output') else str(opp_spaces_output)

#         # Store the results in the state.
#         self.state["opp_spaces_results"] = opp_spaces_data

#         # Return the opportunity spaces data for the next step.
#         return opp_spaces_data

# #OPPORTUNITY IMAGES GENERATION
#     @listen(generate_opportunity_spaces)
#     def generate_opportunity_images(self, opp_spaces_data):
#         # This method runs after the opportunity spaces are generated.
#         # It uses the opp_spaces_data and other context fields to run the opp_images_crew.

#         opp_images_input = {
#             "opportunity_spaces": opp_spaces_data,
#             "sector": my_input["sector"],
#             "target_clients": my_input["target_clients"],
#             "resources": my_input["resources"],
#             "strategic_priorities": my_input["strategic_priorities"],
#             "project_name": my_input["project_name"],
#             "challenge_description": my_input["challenge_description"],
#             "purpose": my_input["purpose"],
#             "focus_constraints": my_input["focus_constraints"]
#         }

#         # Run the opportunity images crew.
#         opp_images_output = opp_images_crew.kickoff(opp_images_input)
#         opp_images_data = opp_images_output.raw_output if hasattr(opp_images_output, 'raw_output') else str(opp_images_output)

#         # Store final opportunity images results.
#         self.state["opp_images_results"] = opp_images_data

#         # Return this final piece of output (images).
#         return opp_images_data

#     async def kickoff_async(self):
#         # This method overrides the default asynchronous kickoff of the Flow.
#         # It runs the entire flow asynchronously and returns a dictionary of final results.

#         result = await super().kickoff_async()
#         return {
#             "patents_data": self.state.get("patents_crew_results"),
#             "scholar_data": self.state.get("scholar_crew_results"),
#             "insights_data": self.state.get("insights_crews_results"),
#             "opp_spaces_data": self.state.get("opp_spaces_results"),
#             "opp_images_data": self.state.get("opp_images_results")
#         }


In [ ]:
flow = InsightGen()
# opp_space_flow = OpportunitySpaceGen()

In [ ]:
# #not displaying
# flow.plot()

# from IPython.display import IFrame

# IFrame(src='./crewai_flow.html', width='150%', height=600)

In [ ]:
# #Display flow
# from IPython.display import HTML

# with open('crewai_flow.html', 'r', encoding='utf-8') as f:
#     html_content = f.read()
# display(HTML(html_content))

# Run Crews

In [ ]:
import asyncio

async def run_flow():
  """Helper function to run the flow using the existing loop."""
  return await flow.kickoff_async()

# Get the current event loop or create a new one if it doesn't exist.
loop = asyncio.get_event_loop()

In [ ]:
result = await flow.kickoff_async() #should be await flow.kickoff()

Streaming output truncated to the last 5000 lines.
      "snippet": "A dry sinsemilla cannabis inflorescence comprising: a) a terpene profile in which myrcene is not the dominant terpene; b) a terpene oil content greater than 1.5%; and c) at least one propyl locus A allele (A pr ); wherein the terpene oil content is the additive content of the terpenes in the terpene \u2026",
      "link": "https://patents.google.com/patent/EP3552482B1/en",
      "priorityDate": "2013-10-29",
      "filingDate": "2014-07-15",
      "grantDate": "2022-06-29",
      "publicationDate": "2022-06-29",
      "inventor": "Mark Anthony Lewis",
      "assignee": "Biotech Institute, LLC",
      "publicationNumber": "EP3552482B1",
      "language": "en",
      "thumbnailUrl": "https://patentimages.storage.googleapis.com/af/30/f7/38e44685b2465b/imgf0001.png",
      "pdfUrl": "https://patentimages.storage.googleapis.com/e5/f9/84/f932cf6cf38de1/EP3552482B1.pdf",
      "figures": [
        {
          "imageUrl": "ht

In [ ]:
result

## Display results

In [ ]:
from IPython.display import Markdown, display

display(Markdown(result["scholar_data"]))

In [ ]:
from IPython.display import Markdown, display

display(Markdown(result["patents_data"]))

In [ ]:
display(Markdown(result["trends_data"]))

In [ ]:
display(Markdown(result["news_data"]))

In [ ]:
display(Markdown(result["customers_data"]))

In [ ]:
display(Markdown(result["competitors_data"]))

In [ ]:
from IPython.display import Markdown, display

display(Markdown(result["insights_data"]))

In [ ]:
display(Markdown(result["opp_spaces_data"]))

In [ ]:
display(Markdown(result["opp_images_data"]))

In [ ]:
# Adding relevant titles and combining the data
combined_markdown = (
    "## Patent Data\n\n" +
    result["patents_data"] +
    "\n\n---\n\n" +
    "## Scholar Data\n\n" +
    result["scholar_data"] +
    "\n\n---\n\n" +
    "## Insights Data\n\n" +
    result["insights_data"] +
    "\n\n---\n\n" +
    "## Opportunity Spaces\n\n" +
    result["opp_spaces_data"] +
    "\n\n---\n\n" +
    "## Opportunity Images\n\n" +
    result["opp_images_data"]
)

# Save to a Markdown file
write_to_file(combined_markdown, directory='/content/drive/MyDrive/Creative_AI/TheBox/Backend_Colab/Outputs', filename='Strauss_1.md')
